In [1]:
from tigramite.independence_tests import cmiknn
import pandas as pd
import numpy as np
import pingouin as pg

C:\Users\succa\anaconda3\lib\site-packages\tigramite\independence_tests\gpdc.py:27: UserWarning: [Errno 2] No such file or directory: 'C:\\Users\\succa\\anaconda3\\lib\\site-packages\\tigramite\\independence_tests\\..\\..\\versions.py'
  warnings.warn(str(e))
C:\Users\succa\anaconda3\lib\site-packages\tigramite\independence_tests\gpdc_torch.py:33: UserWarning: No module named 'gpytorch'
  warnings.warn(str(e))


In [2]:
def CMI_Knn_test(data, x, y, z, Kn, SN):
    X = data[:,x].reshape(data.shape[0],1)
    Y = data[:,y].reshape(data.shape[0],1)
    if len(z)==0:
        Z = None
    else:
        Z = data[:,z]

    cmi = cmiknn.CMIknn(knn=Kn, shuffle_neighbors=SN, sig_samples=500,
                        significance='shuffle_test', transform='ranks')

    MI, p_value = cmi.run_test_raw(X, Y, z=Z)
    return MI, p_value

C:\Users\succa\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
def Test(df, Kn, SN):
    data = np.array(df)
    data_df = pd.DataFrame(data)
    cols = df.columns
    testing_sets = [[0,1,[]], 
                    [0,2,[]],
                    [1,2,[]],
                    [0,1,[2]],
                    [0,2,[1]],
                    [1,2,[0]]]
    for sets in testing_sets:
        x, y, z = sets[0],sets[1],sets[2]
        MI, p_value = CMI_Knn_test(data, x, y, z, Kn, SN)

        parr_corr = pg.partial_corr(data=data_df, x=x, y=y, covar=z, method = "spearman")
        Spear_p_value = parr_corr.iloc[0]["p-val"]
        r = parr_corr.iloc[0]["r"]
        
        MI = round(MI,3)
        p_value = round(p_value,3)

        r = round(r,3)
        Spear_p_value = round(Spear_p_value,3)
        
        if not sets[2]:
            print(cols[sets[0]],cols[sets[1]], MI, r, p_value, Spear_p_value)
        else:
            print(cols[sets[0]],cols[sets[1]],cols[sets[2]][0], MI, r, p_value, Spear_p_value)

In [5]:
df = pd.read_csv("Clean_data/SAMIs_Type_06.csv")[["Firearms","Firearm Licenses","Homicides"]]

In [6]:
Test(df, 0.15, 10) 

Firearms Firearm Licenses 0.048 0.265 0.0 0.0
Firearms Homicides 0.106 0.4 0.0 0.0
Firearm Licenses Homicides 0.005 0.003 0.292 0.93
Firearms Firearm Licenses Homicides 0.041 0.288 0.0 0.0
Firearms Homicides Firearm Licenses 0.09 0.413 0.0 0.0
Firearm Licenses Homicides Firearms 0.007 -0.116 0.032 0.001
